<a href="https://colab.research.google.com/github/Gift-the-glitch/Face-Recognition-Attendance-Marking-System/blob/main/Facial_Recognition_Attendance_Marking_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import face_recognition as fr
import cv2
from datetime import datetime
import tkinter as tk
import os

# Dependency Note:
# Run the following command to install dependencies if not already installed:
# pip install face_recognition numpy opencv-python pillow

# Load and encode known faces - UPDATE PATHS to your own images
def load_known_faces():
    image_paths = [
        "path/to/image1.jpg",
        "path/to/image2.jpg",
        "path/to/image3.jpg",
        "path/to/image4.jpg"
    ]
    known_face_encodings = []
    known_face_names = ["Steve Jobs", "Lex Fridman", "Elon Musk", "Gifty Nelson"]

    for path in image_paths:
        try:
            image = fr.load_image_file(path)
            encoding = fr.face_encodings(image)[0]
            known_face_encodings.append(encoding)
        except IndexError:
            print(f"No face found in {path}. Check the image.")
        except FileNotFoundError:
            print(f"File not found: {path}")
            return [], []

    return known_face_encodings, known_face_names

# Initialize known faces
known_face_encodings, known_face_names = load_known_faces()

# Upload File Functionality
def upload_file():
    global img, filename
    f_types = [('jpg Files', '*.jpg'), ('png Files', '*.png')]
    filename = filedialog.askopenfilename(filetypes=f_types)
    if filename:
        img = ImageTk.PhotoImage(file=filename)
        b2 = tk.Button(root, image=img)
        b2.pack()

# Mark Attendance Functionality
def mark_attendance(name):
    attendance_file = 'attendance_list.csv'
    try:
        with open(attendance_file, 'r+') as f:
            attendance_list = [line.split(',')[0] for line in f.readlines()]
            if name not in attendance_list:
                now = datetime.now()
                dt_string = now.strftime('%d/%b/%Y, %H:%M:%S')
                f.writelines(f'\n{name},{dt_string}')
    except FileNotFoundError:
        print(f"Attendance file not found at {attendance_file}. Please check the path.")

# Process Uploaded Image for Face Recognition
def upload_picture():
    if not filename:
        print("No file selected.")
        return

    frame = cv2.imread(filename)
    frame = cv2.resize(frame, (800, 800))
    rgb_frame = frame[:, :, ::-1]
    face_locations = fr.face_locations(rgb_frame)
    face_encodings = fr.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = fr.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown Entity"
        face_distances = fr.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 1)
        mark_attendance(name)
    cv2.imshow('Uploaded Picture Face Recognition', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Live Face Recognition with Camera
def live_camera():
    video_capture = cv2.VideoCapture(0)
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        rgb_frame = frame[:, :, ::-1]
        face_locations = fr.face_locations(rgb_frame)
        face_encodings = fr.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = fr.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown Entity"
            face_distances = fr.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 1)
            mark_attendance(name)

        cv2.imshow('Live Webcam Face Recognition', frame)

        # Press 'q' to quit live recognition
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

# Stop the Tkinter App
def stop():
    root.destroy()

# Tkinter GUI Setup
root = tk.Tk()
root.title("Face Recognition System")
canvas = tk.Canvas(root, width=500, height=500, bg='skyblue')
canvas.pack()

# GUI Elements
label = tk.Label(root, text='Face Recognition System', font=('Arial', 16))
canvas.create_window(250, 50, window=label)

button_upload = tk.Button(root, text='Upload Image', command=upload_file, bg='green')
canvas.create_window(250, 150, window=button_upload)

button_submit = tk.Button(root, text='Recognize Uploaded Image', command=upload_picture, bg='green')
canvas.create_window(250, 200, window=button_submit)

button_live = tk.Button(root, text='Live Recognition', command=live_camera, bg='green')
canvas.create_window(250, 250, window=button_live)

button_stop = tk.Button(root, text='Exit', command=stop, bg='red')
canvas.create_window(250, 300, window=button_stop)

root.mainloop()

